In [17]:
 import pandas as pd

In [5]:
df_sentiment = pd.read_csv('C:/Users/sharv/Documents/Post Graduate Program In Data Science-PurdueSimpliLearn/Course2-Data Science With Python/Datasets/imdb_labelled.txt',sep='\t',names=['comment','label'])

In [6]:
df_sentiment.head(10)

,comment,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
5,"The rest of the movie lacks art, charm, meanin...",0
6,Wasted two hours.,0
7,Saw the movie today and thought it was a good ...,1
8,A bit predictable.,0
9,Loved the casting of Jimmy Buffet as the scien...,1


In [7]:
# view more information about sentiment data using describe method
df_sentiment.describe()

,label
count,748.000000
mean,0.516043
std,0.500077
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [8]:
# view more info on data
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  748 non-null    object
 1   label    748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [9]:
# view data using groupby and describe method
df_sentiment.groupby('label').describe()

comment                                              
        count unique                               top freq
label                                                      
0         362    361                Not recommended.      2
1         386    384  Definitely worth checking out.      2

In [11]:
# verify the length of the messages and also add it as a new column (feature)
df_sentiment['length'] = df_sentiment['comment'].apply(len)

In [12]:
# view first5 messages with length
df_sentiment.head()

,comment,label,length
0,"A very, very, very slow-moving, aimless movie ...",0,87
1,Not sure who was more lost - the flat characte...,0,99
2,Attempting artiness with black & white and cle...,0,188
3,Very little music or anything to speak of.,0,44
4,The best scene in the movie was when Gerardo i...,1,108


In [13]:
# view first 
df_sentiment[df_sentiment['length']>50]['comment'].iloc[0]

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  '

In [14]:
# start text processing with vectorizer
# start text processing with vectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Use CountVectorizer class to convert the collection of text documents to matrix of tokens, this assigns a numeric value to
# each word present in the text.
vectorizer = CountVectorizer()

In [15]:
# define a function to get rid of stopwords present in the messages
def message_text_process(mess):
    # check characters to see if there r punctuations
    no_punctuation = [char for char in mess if char not in string.punctuation]
    # now form the sentence
    no_punctuation = ''.join(no_punctuation)
    # now eliminate any stopwords
    return[word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]
# example of stopwords are I,me,myself,we our, ours, you,yours. Punctauations are also(like stopwords) less weighted for text analysis.

In [19]:
# verify that the function is working
import string
from nltk.corpus import stopwords
df_sentiment['comment'].head(5).apply(message_text_process)

0    [slowmoving, aimless, movie, distressed, drift...
1    [sure, lost, flat, characters, audience, nearl...
2    [Attempting, artiness, black, white, clever, c...
3                     [little, music, anything, speak]
4    [best, scene, movie, Gerardo, trying, find, so...
Name: comment, dtype: object

In [21]:
# try to convert text into numerical feature vector by using the technique "bag of words".
# bag of words by applying the function and fit the data(comment)into it
bag_of_words = CountVectorizer(analyzer=message_text_process).fit(df_sentiment['comment'])
# created object bag_of_words and then vectorized or assign numeric values to them by fitting the message data using the fit method.

In [22]:
# apply transform method for the bag of words
comment_bagofwords = bag_of_words.transform(df_sentiment['comment'])

In [23]:
# apply tfidf transformer and fit the bag of words into it (transformed version)
# tfidf means term frequency times inverse document frequency 
# this is a common term weighing scheme in information retrievel.First we fit the bag of words into the transformer using 
# TfidfTransformer() instance and then we use the transform method. It actually transforms a count matrix into tfidf representation.
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(comment_bagofwords)

In [27]:
# print the shape of the tfidf
comment_tfidf = tfidf_transformer.transform(comment_bagofwords)
print(comment_tfidf.shape)

(748, 3259)


In [28]:
# choose naive bayes model to detect the sentiment and fit the tfidf data into it
# naive bayes is one of the most statistical procedures or model to classify the text.
from sklearn.naive_bayes import MultinomialNB
sentiment_detection_model = MultinomialNB().fit(comment_tfidf,df_sentiment['label'])

In [30]:
# Once u have fit the transformed tfidf comment or bag of words and response into the model, we can go ahead and try to 
# predict the response for the given comment.
# check model for the predicted and expected value say for comment0 
# first get the 0th comment and then transform that comment as bag of words using the transform function. and then again
# perform tfidf transformation using tfidf_transformer function. Now try to predict the response using predict() method 
# of the model and check actual value present for the first comment in the dataset.
comment = df_sentiment['comment'][0]
bag_of_words_for_comment = bag_of_words.transform([comment])
tfidf = tfidf_transformer.transform(bag_of_words_for_comment)

print('predicted sentiment label',sentiment_detection_model.predict(tfidf)[0])
print('expected sentiment label',df_sentiment.label[0])

predicted sentiment label 0
expected sentiment label 0


In [31]:
comment = df_sentiment['comment'][4]
bag_of_words_for_comment = bag_of_words.transform([comment])
tfidf = tfidf_transformer.transform(bag_of_words_for_comment)

print('predicted sentiment label',sentiment_detection_model.predict(tfidf)[0])
print('expected sentiment label',df_sentiment.label[4])

predicted sentiment label 1
expected sentiment label 1


In [ ]:
# as u can see predicted value matches the actual value. This proves that the text processing algorithm and model r working
# properly.